<a href="https://colab.research.google.com/github/Plabiany/Classification-Images/blob/main/Image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy
from tensorflow.keras.layers import BatchNormalization, LeakyReLU, MaxPooling2D, LSTM, Dense, Dropout, TimeDistributed, Conv2D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D,Conv3D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from tensorflow.keras.applications.vgg19 import VGG19
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from keras.layers import UpSampling2D

def branch_double(size_output, n_feature, size_kernel,x):
    x = Conv2D(n_feature, size_kernel, padding='same', activation='relu', dilation_rate=2)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    return x

def get_model(x):
    n_feature = 64
    size_kernel = (2,2)

    x = UpSampling2D()(x)

    new_x = []
    for i in range(2):
      
      x1 = branch_double(5, n_feature, size_kernel, x)
       
      new_x.append(x1)
    x = Concatenate(axis=1)(new_x)

    return(x)

In [2]:
import time
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from tensorflow.keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.vgg19 import VGG19
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from PIL import Image
import os
import numpy as np
from sklearn.model_selection import train_test_split

img_w = img_h = 64
folder = '/content/drive/MyDrive/dataset'
list_class = os.listdir(folder)
imgs = []
labels = []
for i in range(len(list_class)):

  img_names = os.listdir(os.path.join(folder,list_class[i]))
  img_names.sort()
  for j in range(len(img_names)):

    img = Image.open(os.path.join(folder,list_class[i] + '/' +img_names[j]))
    img = img.resize((img_w,img_h))
    img = np.array(img)
    if len(img.shape) == 2:
      img = np.repeat(img[:, :, np.newaxis], 3, axis=2) / 255

    imgs.append(img[:,:,0:3])
    
    labels.append([i])
  

imgs = np.array(imgs)

print(imgs.shape)
print(imgs.shape)
labels = np.array(labels)
print(labels.shape)

X_train, X_test, y_train, y_test = train_test_split(imgs, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train)
print(y_train.shape)
y_test = np_utils.to_categorical(y_test)
print(y_test.shape)


base_model = VGG19(include_top=False, 
             input_shape=(img_w,img_h,3),weights='imagenet', classes=5)
x = base_model.get_layer('block5_pool').output
x = get_model(x)
x = Flatten(name='Flatten')(x)
x = Dense(64, activation='relu', name='fc1')(x)
predictions = Dense(5, activation='softmax')(x)
model = keras.Model(inputs=base_model.input, outputs=predictions)

epochs = 100
lrate = 0.001
decay = lrate/epochs
sgd = SGD(learning_rate=lrate, momentum=0.9, decay=decay, nesterov=False)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
plot_model(model=model, show_shapes=True)

batch_size = 64
# Fit the model

start = time.time()
model.fit(X_train, y_train, validation_split=0.33 , epochs=epochs, batch_size=32)
end = time.time()
print('time',end-start)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

y_pred = model.predict(X_test)
m = keras.metrics.MeanIoU(num_classes=5)
m.update_state(y_pred, y_test)

y_pred_norma = []
y_test_norma = []
for p in range(len(y_pred)):
  m = max(y_pred[p])
  y_pred_norma.append([i for i, j in enumerate(y_pred[p]) if j == m])

  m1 = max(y_test[p])
  y_test_norma.append([i for i, j in enumerate(y_test[p]) if j == m1])

print(y_pred_norma)
print(y_test_norma)

print('acc',accuracy_score(y_test_norma, y_pred_norma))
print('cohen_kappa_score', cohen_kappa_score(y_test_norma,y_pred_norma))

from sklearn.metrics import precision_score, recall_score, f1_score
print('precision',precision_score(y_test_norma,y_pred_norma, average='macro'))
print('recall',recall_score(y_test_norma,y_pred_norma, average='macro'))
print('f1',f1_score(y_test_norma,y_pred_norma, average='macro'))
confusion_matrix(y_test_norma, y_pred_norma, labels=["0", "1", "2", '3','4'])


(329, 64, 64, 3)
(329, 64, 64, 3)
(329, 1)
(263, 64, 64, 3) (66, 64, 64, 3) (263, 1) (66, 1)
(263, 5)
(66, 5)
80150528/80134624 [==============================] - 0s 0us/step
Epoch 1/100
6/6 [==============================] - 16s 825ms/step - loss: 2.1987 - accuracy: 0.2216 - val_loss: 1.5565 - val_accuracy: 0.3218
Epoch 2/100
6/6 [==============================] - 0s 68ms/step - loss: 1.3295 - accuracy: 0.4943 - val_loss: 1.5590 - val_accuracy: 0.3218
Epoch 3/100
6/6 [==============================] - 0s 69ms/step - loss: 1.0423 - accuracy: 0.6080 - val_loss: 1.5066 - val_accuracy: 0.4023
Epoch 4/100
6/6 [==============================] - 0s 71ms/step - loss: 0.6196 - accuracy: 0.8523 - val_loss: 1.4505 - val_accuracy: 0.4023
Epoch 5/100
6/6 [==============================] - 0s 70ms/step - loss: 0.4814 - accuracy: 0.8580 - val_loss: 1.4750 - val_accuracy: 0.4138
Epoch 6/100
6/6 [==============================] - 0s 70ms/step - loss: 0.3317 - accuracy: 0.8920 - val_loss: 1.3716 - val_

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [11]:
!nvidia-smi

Sun Apr 10 15:44:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

CPU RAM Free: 12.7 GB
GPU 0 ... Mem Free: 16280MB / 16280MB | Utilization   0%


In [11]:
import time
from PIL import Image
import numpy as np
# img = Image.open('/content/skol_na_mesa.jpg')
img = Image.open('/content/bud_4.jpg')
img = img.resize((img_h,img_w))
img = np.array(img)
print(img.shape)
img = np.expand_dims(img, axis=0)
print(img.shape)
start = time.time()

pred = model.predict(img)

end = time.time()
print('time',end-start)
m = np.max(pred)
display([i for i, j in enumerate(pred[0]) if j == m])

(32, 32, 3)
(1, 32, 32, 3)
time 0.34937500953674316


[1]